### 加载Overlay

In [49]:
from pynq import Overlay
overlay = Overlay("./md5.bit")
md5 = overlay.md5_wrap_0
register_map = md5.s_axi_CTRL.register_map

### 给ip分配内存

In [50]:
from pynq import allocate
text_length=1024
input_buffer=allocate(shape=(1024,), dtype='uint8',cacheable=False)
output_buffer=allocate(shape=(16,), dtype='uint8',cacheable=False)

### 配置ip
`pynq.allocate`函数用于为PL中的IP分配可以使用的内存空间。
- 查看vitis hls的Synthesis Summery可知，text_length的Offset为0x18，text_input_1为0x10，result_1为0x1c。
- 使用IP的write方法，将刚分配的内存空间的物理地址写入到ip对应位置上。

In [51]:
import hashlib
import array
import math
import numpy as np
import time

# 硬件加速函数
def MD5_hardware(bstr):
    # 把bstr从python的内存空间复制到连续内存空间中
#     input_buffer[:] = np.zeros(1024, dtype='uint8')# 清空input_buffer
    input_buffer[:len(bstr)] = bytearray(bstr)
    
    # 给ip传参
    md5.s_axi_CTRL.write(0x18,len(bstr))
    md5.s_axi_control.write(0x10,input_buffer.physical_address)
    md5.s_axi_control.write(0x1c,output_buffer.physical_address)

    # 启动ip
    register_map.CTRL.AP_START = 1
    
    # 等待运算结束
    while (register_map.CTRL.AP_DONE == 0):
        pass
    
    # 将结果转换为16进制string
    result = bytearray(output_buffer)
    return result.hex()

# 软件库函数
def MD5_software(bstr):
    return hashlib.md5(bstr).hexdigest()

### 测试用例
测试结果表明我们所设计的硬件加速函数功能等效于软件库函数。

In [52]:
import secrets

print(MD5_hardware(b'123456')) # 输出结果应该为"e10adc3949ba59abbe56e057f20f883e"
print(MD5_software(b'abcdefghijklmn'))# 输出结果应该为"0845a5972cd9ad4a46bad66f1253581f"

for i in range(0, 512+1):
    random_bytes = secrets.token_bytes(i)# secrets.token_bytes(i)返回i个随机字节，适用于密码学安全
    hw = MD5_hardware(random_bytes)
    sw = MD5_software(random_bytes)
    assert (hw == sw), (i, hw, sw)

print("All tests passed")

e10adc3949ba59abbe56e057f20f883e
0845a5972cd9ad4a46bad66f1253581f
All tests passed


### 计算加速比

In [53]:
sw_time = %timeit -n 1000 -r 5 -o MD5_software(secrets.token_bytes(16))
hw_time = %timeit -n 1000 -r 5 -o MD5_hardware(secrets.token_bytes(16))
print('Performance gain:', sw_time.average / hw_time.average) 

23.6 µs ± 2.23 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)
482 µs ± 19.5 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)
Performance gain: 0.04886471362146024
